# 1 Objetivo

O presente notebook tem como objetivo explorar a forma de apuramento do número de deputados eleitos por círculo eleitoral utilizado no sistema eleitoral português - **Método da média mais alta de Hondt** (./dados_oficiais/sistema_eleitoral.pdf).

Este documento encontra-se dividido em 2 secções:

- Replicação de exemplo: Aqui tenta-se replicar a forma de cálculo do exemplo apresentado mais abaixo
- Resultados 2022: Aplicação da forma de cálculo aos resultados de 2022

FONTE: https://www.parlamento.pt/Parlamento/Paginas/SistemaEleitoral.aspx

**Método da média mais alta de Hondt**

Fórmula de cálculo, criada pelo advogado belga Victor D' Hondt, utilizada na distribuição de mandatos pelos candidatos das listas concorrentes a eleições, com base no princípio da representação proporcional. Consiste na repartição dos mandatos pelos partidos, proporcionalmente à importância da respetiva votação.

``Exemplo prático``

O círculo eleitoral "x" tem direito a eleger 7 deputados. Concorrem 4 partidos A, B, C, D.
Número de votos por partido: A - 12 000 | B - 7500 | C - 4500 | D - 3000.

**Aplicação da regra**

**1.º O número de votos apurados por cada lista é dividido, sucessivamente, por 1, 2, 3, 4, etc. (até 7, que é o número de Deputados a eleger), sendo os quocientes alinhados por ordem decrescente:**

**divisão por 1**  | A: 12 000 | B 7500 | C: 4500 | D: 3000<br>
**divisão por 2**  | A: 6000 | B: 3750 | C: 2250 | D: 1500<br>
**divisão por 3**   | A: 4000 | B: 2500 | C: 1500 | D: 1000<br>
**divisão por 4**   | A: 3000 | B: 1875 | C: 1125 | D: 750<br>

(...)

O último mandato, existindo quociente igual nas listas A e D, é atribuído à lista D, por ser esta a que tem menor número de votos.

**2.º Ordenam-se os quocientes, atribuindo-se desta forma os mandatos:**

1.º Deputado - partido A<br>
2.º Deputado - partido B<br>
3.º Deputado - partido A<br>
4.º Deputado - partido C<br>
5.º Deputado - partido A<br>
6.º Deputado - partido B<br>
7.º Deputado - partido D<br>

Total

Partido A - 3 deputados<br>
Partido B - 2 deputados<br>
Partido C - 1 deputado<br>
Partido D - 1 deputado

# 2 Replicação de exemplo

In [81]:
import pandas as pd

# Cria um DataFrame com os votos de cada partido
df_sistema_eleitoral = pd.DataFrame([12000, 7500, 4500, 3000], index=['Partido A', 'Partido B', 'Partido C', 'Partido D'], columns=['votos'])

# Crie um dicionário para armazenar os resultados do método de Hondt
dic = {}

# Itere sobre as linhas do DataFrame
for i, v in df_sistema_eleitoral.iterrows():
    votos = v[0] 
    
    # Calcule os resultados do método de Hondt para cada partido tendo como base 7 deputados a eleger
    dic[i] = [votos // x for x in range(1, 8)]
    
# Crie um novo DataFrame com os resultados do método de Hondt
df_sistema_eleitoral_hont = pd.DataFrame(dic)
df_sistema_eleitoral_hont

,Partido A,Partido B,Partido C,Partido D
0,12000,7500,4500,3000
1,6000,3750,2250,1500
2,4000,2500,1500,1000
3,3000,1875,1125,750
4,2400,1500,900,600
5,2000,1250,750,500
6,1714,1071,642,428


In [82]:
df_eleitos_se = pd.DataFrame(df_sistema_eleitoral_hont.stack(), columns=['votos']).reset_index().sort_values('votos', ascending=False).head(7)
df_eleitos_se


,level_0,level_1,votos
0,0,Partido A,12000
1,0,Partido B,7500
4,1,Partido A,6000
2,0,Partido C,4500
8,2,Partido A,4000
5,1,Partido B,3750
3,0,Partido D,3000


In [83]:
df_eleitos_se.groupby('level_1')['votos'].count().sort_values(ascending = False)

level_1
Partido A    3
Partido B    2
Partido C    1
Partido D    1
Name: votos, dtype: int64

# 3 Análise resultados 2022

In [84]:
df_resultados = pd.read_json('../data/resultados_2022.json')
df_resultados.head()

,Partido,Aveiro,Beja,Braga,Bragança,Castelo Branco,Coimbra,Évora,Faro,Guarda,...,Santarém,Setúbal,Viana do Castelo,Vila Real,Viseu,RA Açores,RA Madeira,Europa,Fora da Europa,Total
0,A,0,0,286,0,0,320,0,0,0,...,0,0,173,0,0,0,0,84,479,2467
1,ADN,677,0,0,0,176,0,0,1157,0,...,0,1310,0,0,188,486,548,262,648,10874
2,BE,16706,2511,18550,1370,4069,10888,2629,11209,2368,...,10011,24940,4418,2442,5218,3613,4120,2647,1699,244603
3,CDS,8966,515,8216,1373,1502,3244,896,2111,1716,...,4137,4863,4273,1665,3789,0,0,1156,1379,89181
4,CH,20570,6932,28752,5610,7959,13182,7221,24000,6134,...,23813,39171,7702,7569,14383,4991,7730,7756,6123,399659


In [85]:
df_num_deputados = pd.read_json('../data/numero_deputados_2022.json')
df_num_deputados['circulo'] = df_num_deputados['circulo'].str.replace('Madeira', 'RA Madeira')
df_num_deputados['circulo'] = df_num_deputados['circulo'].str.replace('Açores', 'RA Açores')
dic_num_deputados = df_num_deputados.set_index('circulo').to_dict()['nr_deputados']
dic_num_deputados

{'Aveiro': 16,
 'Beja': 3,
 'Braga': 19,
 'Bragança': 3,
 'Castelo Branco': 4,
 'Coimbra': 9,
 'Évora': 3,
 'Faro': 9,
 'Guarda': 3,
 'Leiria': 10,
 'Lisboa': 48,
 'Portalegre': 2,
 'Porto': 40,
 'Santarém': 9,
 'Setúbal': 18,
 'Viana do Castelo': 6,
 'Vila Real': 5,
 'Viseu': 8,
 'RA Madeira': 6,
 'RA Açores': 5,
 'Europa': 2,
 'Fora da Europa': 2}

In [86]:
dic_eleitos = {}

for k, v in dic_num_deputados.items():
    circulo = k
    nr_deputados = v

    dic = {}

    df_eleitos_res = df_resultados[['Partido', circulo]].loc[df_resultados[circulo] != 0]

    for i, v in df_eleitos_res.iterrows():
        partido = v[0]
        quocientes = [ v[1]/x for x in range(1, nr_deputados + 1)]
        dic[partido] = quocientes

        df_resultados_hondt = pd.DataFrame(dic)

        df_eleitos_res = pd.DataFrame(df_resultados_hondt.stack(), columns=['votos']).reset_index().sort_values('votos', ascending=False).head(nr_deputados)

    dic_eleitos[k] = df_eleitos_res.groupby('level_1')['votos'].count().sort_values(ascending = False).to_dict()
    

display(pd.DataFrame(dic_eleitos))
print(dic_eleitos)

,Aveiro,Beja,Braga,Bragança,Castelo Branco,Coimbra,Évora,Faro,Guarda,Leiria,...,Porto,Santarém,Setúbal,Viana do Castelo,Vila Real,Viseu,RA Madeira,RA Açores,Europa,Fora da Europa
PS,8.0,2.0,9.0,2.0,3.0,6.0,2.0,5.0,2.0,5.0,...,19.0,5.0,10.0,3.0,3.0,4.0,3.0,3.0,2.0,1.0
PPD/PSD,7.0,NaN,8.0,1.0,1.0,3.0,1.0,3.0,1.0,4.0,...,14.0,3.0,3.0,3.0,2.0,4.0,NaN,NaN,NaN,1.0
CH,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,...,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PCP/PEV/CDU,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IL,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PPD/PSD/CDS-PP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
PPD/PSD/CDS-PP/PPM-AD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN


{'Aveiro': {'PS': 8, 'PPD/PSD': 7, 'CH': 1}, 'Beja': {'PS': 2, 'PCP/PEV/CDU': 1}, 'Braga': {'PS': 9, 'PPD/PSD': 8, 'CH': 1, 'IL': 1}, 'Bragança': {'PS': 2, 'PPD/PSD': 1}, 'Castelo Branco': {'PS': 3, 'PPD/PSD': 1}, 'Coimbra': {'PS': 6, 'PPD/PSD': 3}, 'Évora': {'PS': 2, 'PPD/PSD': 1}, 'Faro': {'PS': 5, 'PPD/PSD': 3, 'CH': 1}, 'Guarda': {'PS': 2, 'PPD/PSD': 1}, 'Leiria': {'PS': 5, 'PPD/PSD': 4, 'CH': 1}, 'Lisboa': {'PS': 21, 'PPD/PSD': 13, 'CH': 4, 'IL': 4, 'BE': 2, 'PCP/PEV/CDU': 2, 'L': 1, 'PAN': 1}, 'Portalegre': {'PS': 2}, 'Porto': {'PS': 19, 'PPD/PSD': 14, 'BE': 2, 'CH': 2, 'IL': 2, 'PCP/PEV/CDU': 1}, 'Santarém': {'PS': 5, 'PPD/PSD': 3, 'CH': 1}, 'Setúbal': {'PS': 10, 'PPD/PSD': 3, 'PCP/PEV/CDU': 2, 'BE': 1, 'CH': 1, 'IL': 1}, 'Viana do Castelo': {'PPD/PSD': 3, 'PS': 3}, 'Vila Real': {'PS': 3, 'PPD/PSD': 2}, 'Viseu': {'PPD/PSD': 4, 'PS': 4}, 'RA Madeira': {'PPD/PSD/CDS-PP': 3, 'PS': 3}, 'RA Açores': {'PS': 3, 'PPD/PSD/CDS-PP/PPM-AD': 2}, 'Europa': {'PS': 2}, 'Fora da Europa': {'PPD/P